In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor


In [ ]:

train=pd.read_csv("../input/train_V2.csv")
test=pd.read_csv("../input/test_V2.csv")

In [ ]:
train=train.dropna()

In [ ]:
'''def describe_categorical(X):
    from IPython.display import display,HTML
    display(HTML(X[X.columns[X.dtypes=="object"]].describe().to_html()))

describe_categorical(train)'''


In [ ]:
test_id = test['Id']
dropfeature=['groupId','matchId','Id']
train.drop(dropfeature,axis=1,inplace=True)
test.drop(dropfeature,axis=1,inplace=True)

In [ ]:
Ohe_col=['matchType']

for variable in Ohe_col:
    dummies=pd.get_dummies(train[variable],prefix=variable)
    train=pd.concat([train,dummies],axis=1)
    train.drop([variable],axis=1,inplace=True)

In [ ]:
for variable in Ohe_col:
    dummies=pd.get_dummies(test[variable],prefix=variable)
    test=pd.concat([test,dummies],axis=1)
    test.drop([variable],axis=1,inplace=True)

In [ ]:
Outliers=list(train.columns[train.dtypes!='object'])

for variable in Outliers:
    quartile_1, quartile_3 = np.percentile(train[variable], [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    train = train.drop(train[(train[variable] > upper_bound) | (train[variable]< lower_bound)].index)

In [ ]:
y = train['winPlacePerc'] 
train.drop('winPlacePerc', axis = 1, inplace = True)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,y,test_size = 0.1,random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc_x = MinMaxScaler((-1,1))
x_train  = sc_x.fit_transform(X_train)
x_test = sc_x.transform(X_test)

In [ ]:
rfr=RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=None, min_samples_split=2,
min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)

gbr=GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, subsample=1.0, 
                              criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, 
                              min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0,
                              min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9,
                              verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto')

dtr= DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=None, min_samples_split=2,
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None,
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, presort=False)

models=[rfr,gbr,dtr]

for model in models:
    model.fit(X_train,y_train)
    predict = model.predict(X_test)
    print(mean_absolute_error(y_test,predict))
    print('****************\n')
    


In [ ]:
rfr.fit(train,y)
prediction = rfr.predict(test)

In [ ]:
submission=pd.DataFrame(
            {
                'Id': test_id,
                'winPlacePerc': prediction
            }

)

submission.to_csv('submitPUBG01.csv',index=False)

In [ ]:
submission.head

In [ ]:
submission.shape

In [ ]:
submission=pd.read_csv("submitPUBG01.csv")